# Credit Risk Resampling Techniques

In [374]:
import warnings
warnings.filterwarnings('ignore')

In [375]:

import numpy as np
import pandas as pd
from pathlib import Path
import collections
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Read the CSV and Perform Basic Data Cleaning

In [376]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [377]:
# Load the data
#file_path = Path('LoanStats_2019Q1.csv.zip')
file_path = ('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

pd.set_option('display.max_columns', 500)

df.head()


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,w,10204.87,10204.87,355.29,355.29,295.13,60.16,0.0,0.0,0.0,375.35,May-2019,0.0,1.0,Individual,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,59.0,1.0,2.0,1553.0,58.0,3700.0,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,0.0,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,w,24293.13,24293.13,873.53,873.53,706.87,166.66,0.0,0.0,0.0,929.09,May-2019,0.0,1.0,Individual,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,86.0,4.0,8.0,5197.0,61.0,50400.0,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,0.0,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,w,19636.79,19636.79,485.44,485.44,363.21,122.23,0.0,0.0,0.0,529.88,May-2019,0.0,1.0,Individual,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,75.0,2.0,3.0,2810.0,75.0,17500.0,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,0.0,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,w,9719.34,9719.34,335.33,335.33,280.66,54.67,0.0,0.0,0.0,353.55,May-2019,0.0,1.0,Individual,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,66.0,0.0,1.0,6360.0,69.0,23000.0,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,w,21614.73,21614.73,484.36,484.36,385.27,99.09,0.0,0.0,0.0,520.39,May-2019,0.0,1.0,Individual,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,84.0,1.0,1.0,3600.0,41.0,43100.0,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [378]:
df2=df.drop(['issue_d', 'out_prncp_inv', 'total_pymnt_inv','next_pymnt_d'], axis = 1)
df2.head()
#encode paymnet plan, initial_list_status, application type, hardship flag, debt_settlment_flag

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,w,10204.87,355.29,295.13,60.16,0.0,0.0,0.0,375.35,0.0,1.0,Individual,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,59.0,1.0,2.0,1553.0,58.0,3700.0,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,0.0,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,w,24293.13,873.53,706.87,166.66,0.0,0.0,0.0,929.09,0.0,1.0,Individual,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,86.0,4.0,8.0,5197.0,61.0,50400.0,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,0.0,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,w,19636.79,485.44,363.21,122.23,0.0,0.0,0.0,529.88,0.0,1.0,Individual,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,75.0,2.0,3.0,2810.0,75.0,17500.0,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,0.0,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,w,9719.34,335.33,280.66,54.67,0.0,0.0,0.0,353.55,0.0,1.0,Individual,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,66.0,0.0,1.0,6360.0,69.0,23000.0,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,w,21614.73,484.36,385.27,99.09,0.0,0.0,0.0,520.39,0.0,1.0,Individual,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,84.0,1.0,1.0,3600.0,41.0,43100.0,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [379]:
#encode paymnet plan, initial_list_status, application type, hardship flag, debt_settlment_flag....
le = LabelEncoder()
df3 = df2.copy()
df3['pymnt_plan'] = le.fit_transform(df2['pymnt_plan']) 
df3['initial_list_status'] = le.fit_transform(df2['initial_list_status'])
df3['application_type']= le.fit_transform(df2['application_type'])
df3['hardship_flag']= le.fit_transform(df2['hardship_flag'])
df3['debt_settlement_flag']= le.fit_transform(df2['debt_settlement_flag'])
df3['home_ownership'] = le.fit_transform(df2['home_ownership'])
df3['verification_status'] = le.fit_transform(df2['verification_status'])

df3.tail(10)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68807,35000.0,0.0819,1099.85,3,75005.0,0,low_risk,0,36.34,0.0,1.0,17.0,0.0,27198.0,24.0,0,32399.41,3283.63,2600.59,683.04,0.0,0.0,0.0,1099.85,0.0,1.0,0,0.0,0.0,61362.0,1.0,3.0,1.0,1.0,3.0,34164.0,60.0,0.0,0.0,8478.0,45.0,68400.0,0.0,4.0,1.0,1.0,3610.0,22513.0,53.6,0.0,0.0,82.0,100.0,27.0,3.0,0.0,27.0,3.0,0.0,7.0,8.0,10.0,13.0,6.0,14.0,18.0,8.0,17.0,0.0,0.0,0.0,1.0,100.0,30.0,0.0,0.0,129784.0,61362.0,48500.0,61384.0,0,0
68808,10000.0,0.0819,314.25,1,47000.0,1,low_risk,0,12.51,0.0,0.0,16.0,1.0,4468.0,27.0,1,9256.95,938.20,743.05,195.15,0.0,0.0,0.0,314.25,0.0,1.0,0,0.0,0.0,171524.0,1.0,1.0,0.0,1.0,22.0,2811.0,31.0,1.0,2.0,2293.0,11.0,55200.0,0.0,0.0,1.0,3.0,10720.0,28669.0,12.3,0.0,0.0,152.0,111.0,4.0,4.0,2.0,31.0,9.0,0.0,4.0,7.0,7.0,8.0,4.0,14.0,21.0,7.0,16.0,0.0,0.0,0.0,1.0,100.0,0.0,1.0,0.0,240387.0,7279.0,32700.0,9187.0,0,0
68809,6500.0,0.1502,225.39,1,70000.0,0,low_risk,0,34.78,0.0,1.0,13.0,0.0,11082.0,28.0,0,6210.13,477.91,289.87,188.04,0.0,0.0,0.0,225.39,0.0,1.0,0,0.0,0.0,139066.0,1.0,5.0,2.0,4.0,4.0,38927.0,64.0,0.0,0.0,6875.0,66.0,15100.0,3.0,0.0,4.0,4.0,12642.0,140.0,98.4,0.0,0.0,184.0,296.0,36.0,4.0,4.0,38.0,1.0,0.0,2.0,4.0,2.0,5.0,7.0,7.0,17.0,4.0,13.0,0.0,0.0,0.0,2.0,100.0,100.0,0.0,0.0,184448.0,50009.0,8500.0,60792.0,0,0
68810,9600.0,0.0756,298.89,1,64000.0,0,low_risk,0,14.27,0.0,1.0,13.0,0.0,6881.0,17.0,1,8880.25,892.64,719.75,172.89,0.0,0.0,0.0,298.89,0.0,1.0,0,0.0,0.0,200024.0,2.0,2.0,0.0,1.0,23.0,5334.0,26.0,3.0,6.0,2508.0,17.0,51000.0,3.0,0.0,2.0,7.0,15386.0,38619.0,15.1,0.0,0.0,33.0,64.0,1.0,1.0,1.0,1.0,3.0,0.0,6.0,6.0,9.0,12.0,2.0,10.0,14.0,6.0,13.0,0.0,0.0,0.0,3.0,100.0,11.1,0.0,0.0,278473.0,12215.0,45500.0,20412.0,0,0
68811,10000.0,0.0646,306.31,1,98000.0,1,low_risk,0,22.41,1.0,0.0,16.0,0.0,7581.0,39.0,1,9238.49,911.75,761.51,150.24,0.0,0.0,0.0,306.31,0.0,1.0,0,0.0,0.0,256247.0,0.0,3.0,0.0,2.0,22.0,20699.0,36.0,0.0,4.0,2058.0,22.0,70800.0,2.0,0.0,0.0,6.0,16015.0,43528.0,10.8,0.0,0.0,170.0,180.0,13.0,13.0,2.0,20.0,22.0,0.0,4.0,9.0,6.0,19.0,11.0,12.0,26.0,9.0,16.0,0.0,0.0,0.0,0.0,92.1,0.0,0.0,0.0,385928.0,28280.0,48800.0,57128.0,0,0
68812,10000.0,0.1502,346.76,3,26000.0,1,low_risk,0,9.60,0.0,0.0,9.0,0.0,2684.0,20.0,1,9326.86,1019.42,673.14,346.28,0.0,0.0,0.0,346.76,0.0,1.0,0,0.0,242.0,6798.0,1.0,3.0,0.0,0.0,160.0,4114.0,76.0,3.0,6.0,977.0,33.0,15200.0,0.0,1.0,1.0,6.0,755.0,9677.0,14.4,0.0,0.0,185.0,17.0,4.0,4.0,0.0,4.0,3.0,4.0,3.0,5.0,3.0,3.0,14.0,6.0,6.0,5.0,9.0,0.0,0.0,0.0,3.0,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,0,0
68813,12000.0,0.2727,368.37,3,63000.0,0,low_risk,0,29.07,0.0,0.0,8.0,0.0,13314.0,26.0,1,11706.42,1086.93,293.58,793.35,0.0,0.0,0.0,368.37,0.0,1.0,0,0.0,0.0,60350.0,3.0,4.0,2.0,2.0,2.0,47036.0,75.0,3.0,5.0,1177.0,69.0,25000.0,1.0,0.0,4.0,7.0,7544.0,11387.0,15.7,0.0,0.0,164.0,107.0,1.0,1.0,0.0,1.0,2.0,0.0,3.0,4.0,3.

In [380]:
pd.set_option('display.max_rows', 90)
df3.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                  int32
annual_inc                    float64
verification_status             int32
loan_status                    object
pymnt_plan                      int32
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status             int32
out_prncp                     float64
total_pymnt                   float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_amnt               float64
collections_12_mths_ex_med    float64
policy_code                   float64
application_

# Split the Data into Training and Testing

In [381]:
#get x & y
#load status is the Y, so drop it for the X dataset
X = df3.drop(columns="loan_status") 

#scale the df 
data_scaler = StandardScaler()
X = data_scaler.fit_transform(X)


y = df3['loan_status'].to_frame() 




In [382]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [383]:
#split into training & tests sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [384]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(sampling_strategy=1,random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [385]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [386]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.704539034352453

In [387]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   66,    35],
       [ 4180, 12924]], dtype=int64)

In [388]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.65      0.76      0.03      0.70      0.49       101
   low_risk       1.00      0.76      0.65      0.86      0.70      0.50     17104

avg / total       0.99      0.76      0.65      0.85      0.70      0.50     17205



### SMOTE Oversampling

In [389]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(sampling_strategy = 1, random_state=1).fit_resample(
    X_train, y_train)
print(sorted(Counter(y_resampled).items()))


[('loan_status', 1)]


In [390]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [391]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6988577161036964

In [392]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   65,    36],
       [ 4205, 12899]], dtype=int64)

In [393]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.64      0.75      0.03      0.70      0.48       101
   low_risk       1.00      0.75      0.64      0.86      0.70      0.49     17104

avg / total       0.99      0.75      0.64      0.85      0.70      0.49     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [404]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
cc = ClusterCentroids(sampling_strategy = 1, random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [405]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [406]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6928201034556215

In [397]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   66,    35],
       [ 4700, 12404]], dtype=int64)

In [398]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.65      0.73      0.03      0.69      0.47       101
   low_risk       1.00      0.73      0.65      0.84      0.69      0.48     17104

avg / total       0.99      0.72      0.65      0.83      0.69      0.48     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [399]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
sm = SMOTEENN(sampling_strategy = 1, random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [400]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [401]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6903902972149414

In [402]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   66,    35],
       [ 4664, 12440]], dtype=int64)

In [403]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.65      0.73      0.03      0.69      0.47       101
   low_risk       1.00      0.73      0.65      0.84      0.69      0.48     17104

avg / total       0.99      0.73      0.65      0.84      0.69      0.48     17205

